In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import json
import cv2
import numpy as np
from tqdm import tqdm, tqdm_notebook
import random
import glob

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss, MSELoss
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F
from numpy.random import seed
import torchvision
from torchvision import transforms # 이미지 데이터 transform

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
if device =='cuda':
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)

print(device)

## data preprocessing

In [ ]:
data_dir = '/kaggle/input/best-artworks-of-all-time/'
data = pd.read_csv('/kaggle/input/best-artworks-of-all-time/artists.csv')

In [ ]:
genre_df = data.groupby('genre')['paintings'].sum().reset_index(name='Count')

In [ ]:
genre_df[genre_df['Count']>=150].reset_index().sort_values(by='Count', ascending = False)

In [ ]:
genre = np.array(genre_df[genre_df['Count']>=200].reset_index()['genre'])
len(genre)

In [ ]:
genre = ['Baroque', 'Cubism', 'High Renaissance',
        'Impressionism', 'Expressionism', 'Northern Renaissance', 'Pop Art', 
         'Post-Impressionism', 'Primitivism', 'Romanticism', 'Surrealism', 'Symbolism']
genre_list = [x for x in genre]

In [ ]:
from collections import defaultdict

genre_artist = defaultdict(list)
for g in genre:
    for i in range(len(data)):
        if data['genre'].iloc[i] == g:
            genre_artist[g].append(data['name'].iloc[i])

In [ ]:
genre_artist
# genre마다 최대 4명 적으면 1명
#4명이 4개, 3명이 2, 2명이 2, 1명이 4

In [ ]:
!mkdir image_dir
for g in genre:
    gg = g.replace(" ", "_")
    print(gg)
    !mkdir image_dir/$gg
    for i in genre_artist[g]:
        name = i.replace(" ", "_")
        if name == 'Albrecht_Dürer':
            !cp -r /kaggle/input/best-artworks-of-all-time/images/images/Albrecht_Du╠êrer/* 'image_dir/'$gg
        else:
            !cp -r '/kaggle/input/best-artworks-of-all-time/images/images/'$name/* 'image_dir/'$gg

In [ ]:
!mv image_dir/High_Renaissance 'image_dir/High Renaissance'
!mv image_dir/Northern_Renaissance 'image_dir/Northern Renaissance'
!mv image_dir/Pop_Art 'image_dir/Pop Art'

In [ ]:
genre_df = data.groupby('genre')['paintings'].sum().reset_index(name='Count')
genre_df = genre_df[genre_df['genre'].isin(genre)].reset_index()
genre_df = genre_df.drop(['index'],axis = 1)
genre_df

### add class weight

In [ ]:
genre_df['class_weight'] = genre_df.Count.sum() / (genre_df.shape[0] * genre_df.Count)
artists_genre =np.array(genre_df['genre'])  
artists_genre=np.unique(artists_genre)
# class_weights = genre_df['class_weight'].to_dict()
class_weights = genre_df['class_weight']
class_weights = torch.FloatTensor(class_weights).to(device)

### check path

In [ ]:
images_dir = 'image_dir'
artists_dirs = os.listdir(images_dir)

In [ ]:
for name in artists_genre:
    if os.path.exists(os.path.join(images_dir, name)):
        print("Found -->", os.path.join(images_dir, name))
    else:
        print("Did not find -->", os.path.join(images_dir, name))

In [ ]:
artists_temp = genre_df.copy()

In [ ]:
genres = [0]*12
for i, genre in enumerate(artists_genre):
    name = artists_genre[i] + '/*'
    genres[i] = glob.glob(os.path.join(images_dir, name))

for genre in genres:
    print(len(genre))

In [ ]:
import glob

def read_img(file_path):
    img_arr = cv2.imread(file_path)
    return cv2.cvtColor(img_arr, cv2.COLOR_BGR2RGB)


img_arrs = []
img_num = range(0, 193)


for i in random.sample(img_num,3):
    img_arrs.append(read_img(genres[0][i]))
    img_arrs.append(read_img(genres[1][i]))
    img_arrs.append(read_img(genres[2][i]))
print("총 {}개의 이미지 ".format(len(img_arrs)))

In [ ]:
rows = 3
columns = 3

fig, axes = plt.subplots(nrows=rows, ncols = columns, figsize=(columns*3, rows*3))

for num in range(1, rows*columns+1):
    fig.add_subplot(rows, columns, num)
    idx = num-1
    
    plt.imshow(img_arrs[idx], aspect='auto')
    plt.xlabel(f'{img_arrs[idx].shape}', fontsize=12)
    
fig.tight_layout()

cols = ['1', '2', '3']
for folder_idx, ax in enumerate(axes[0]):
    ax.set_title(cols[folder_idx])
    
for idx, ax in enumerate(axes.flat):
    ax.set_xticks([])
    ax.set_yticks([])

In [ ]:
len(genres[0])

In [ ]:
import math

len_test_genres = [0]*12
len_valid_genres = [0]*12

for i in range(12):
    len_test_genres[i] = round(len(genres[i])*0.1)
    len_valid_genres[i] = round(len(genres[i])*0.1)
    print(len_test_genres[i], len_valid_genres[i])

In [ ]:
import shutil
def split(img_list, test_count, train_path, test_path, valid_path):
    test_files = []
    valid_files = []
    for i in random.sample(img_list, test_count):
        test_files.append(i)
    
    train_files = [x for x in img_list if x not in test_files]
    
    for i in random.sample(train_files, test_count):
        valid_files.append(i)
    
    train_files = [x for x in train_files if x not in valid_files]
    
    
    for k in train_files:
        shutil.copy(k, train_path)
    for c in test_files:
        shutil.copy(c, test_path)
    for v in valid_files:
        shutil.copy(v, valid_path)    
        
    print('train 폴더 이미지 개수 : {}\ntest,valid 폴더 이미지 개수 : {}, {}'.format(
        len(glob.glob(train_path+'/*')), len(glob.glob(test_path+'/*')), len(glob.glob(valid_path+'/*'))
    ))

In [ ]:
os.mkdir(os.path.join(images_dir, 'train'))
os.mkdir(os.path.join(images_dir, 'test'))
os.mkdir(os.path.join(images_dir, 'valid'))

for i, genre in enumerate(artists_genre):
    try:
        os.mkdir(os.path.join(images_dir, 'train', genre))
        os.mkdir(os.path.join(images_dir, 'test', genre))
        os.mkdir(os.path.join(images_dir, 'valid', genre))
    except:
        continue

In [ ]:
for i, genre in enumerate(artists_genre):
    train_path = os.path.join(images_dir, 'train', genre)
    test_path = os.path.join(images_dir, 'test', genre)
    valid_path = os.path.join(images_dir, 'valid', genre)
#     print(genre , train_path, test_path)
    split(genres[i], len_test_genres[i], train_path, test_path, valid_path)

## data augmentation

In [ ]:
train_path = os.path.join(images_dir, 'train')
test_path = os.path.join(images_dir, 'test')
valid_path = os.path.join(images_dir, 'valid')
dataset_path = 'image_dir'

In [ ]:
from torch.utils.data import Dataset, DataLoader # 데이터 커스터마이징
from PIL import Image # PIL = Python Image Library
import cv2 # albumentation transform을 쓰려면 꼭 이 라이브러리를 이용
import tensorflow as tf

class Custom_Dataset(Dataset):
    def __init__(self, file_path, mode, transform=None):
        self.all_data = sorted(glob.glob(os.path.join(file_path, mode, '*', '*')))
        self.transform = transform

    def __getitem__(self, index):
        if torch.is_tensor(index):        # 인덱스가 tensor 형태일 수 있으니 리스트 형태로 바꿔준다.
            index = index.tolist()

        data_path = self.all_data[index]
        #img = np.array(Image.open(data_path).convert("RGB")) # albumenatation transform을 쓰려면 cv2 라이브러리로 이미지를 읽어야 함
        image=cv2.imread(data_path)
        image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # BGR -> RGB 변환

        # transform 적용
        if self.transform is not None:    
            augmented = self.transform(image=image)
            image = augmented['image'] 
#             image = image/ 255.0
#             image = image.to(torch.float32)

        # 이미지 이름을 활용해 label 부여
        label=[]  
        
        for i, g in enumerate(genre_list):
            if g == data_path.split('/')[2]:
                label = i
        return image, label

    def __len__(self):
        length = len(self.all_data)
        return length

In [ ]:
import albumentations
import albumentations.pytorch

albumentations_resize = albumentations.Compose([
                                                
    albumentations.Resize(224,224), 
    albumentations.Normalize(mean=0, std=1),
    albumentations.pytorch.transforms.ToTensorV2()
    
])

resize_train=Custom_Dataset(dataset_path, 'train', transform=albumentations_resize)
resize_valid=Custom_Dataset(dataset_path, 'valid', transform=albumentations_resize)
resize_test=Custom_Dataset(dataset_path, 'test', transform=albumentations_resize)

In [ ]:
def tensor_img(img):    
    img = img.permute(1,2,0)
    plt.imshow(img)
    
resize_train[488][0].numpy().shape

In [ ]:
resize_train[488][0].permute(1,2,0).shape

In [ ]:
resize_train[488]

In [ ]:
tensor_img(resize_train[488][0])

In [ ]:
import albumentations
import albumentations.pytorch
from torchvision import transforms # 이미지 데이터 transform
from torch.utils.data import DataLoader # 이미지 데이터 로더

albumentations_train = albumentations.Compose([
                                                
    albumentations.Resize(224, 224),   
    albumentations.RandomResizedCrop(224, 224),
    #albumentations.CenterCrop(224,224),
    albumentations.OneOf([
                          albumentations.HorizontalFlip(p=0.8), # p확률로 이미지 좌우 반전
                          albumentations.RandomRotate90(p=0.8), # p확률로 90도 회전
                          albumentations.VerticalFlip(p=0.8) # p확률로 이미지 상하 반전
    ], p=1),

    albumentations.OneOf([
                          albumentations.MotionBlur(p=0.8), # p확률로 이미지를 흐리게(?) 만들어 줌
#                           albumentations.OpticalDistortion(p=0.8), # p확률로 이미지 왜곡
                          albumentations.GaussNoise(p=0.8) # 임의의 noise를 삽입          
    ], p=1),
    # albumentations.Normalize(mean = resize_train_mean, std = resize_train_std),
    albumentations.Normalize(mean=0, std=1),
    albumentations.pytorch.ToTensorV2()
    
])

albumentations_test = albumentations.Compose([
                                                
    albumentations.Resize(224, 224),
    albumentations.Normalize(mean=0, std=1),
    albumentations.pytorch.ToTensorV2()
    
])


trainset=Custom_Dataset(dataset_path, 'train', transform=albumentations_train)
testset=Custom_Dataset(dataset_path, 'test', transform=albumentations_test)
validset=Custom_Dataset(dataset_path, 'valid', transform=albumentations_test)

train_loader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=0)

test_loader = torch.utils.data.DataLoader(testset, batch_size=16,
                                         shuffle=False, num_workers=0)

valid_loader = torch.utils.data.DataLoader(validset, batch_size=16,
                                         shuffle=False, num_workers=0)

In [ ]:
plt.imshow(resize_train[80][0].permute(1,2,0))

In [ ]:
tensor_img(trainset[80][0])

In [ ]:
resize_train[600][0].dtype

In [ ]:
import torchvision.models.resnet as resnet
import torch.nn as nn
import torch.optim as optim

conv1x1 = resnet.conv1x1
Bottleneck = resnet.Bottleneck
BasicBlock = resnet.BasicBlock

In [ ]:
class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000, zero_init_residual=True):
        super(ResNet, self).__init__()
        self.inplanes = 64 #

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.layer1 = self._make_layer(block, 64, layers[0], stride=1) # 3 반복
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2) # 4 반복
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2) # 6 반복
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2) # 3 반복
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1): # planes -> 입력되는 채널 수
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion: 
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        # input [32, 128, 128] -> [C ,H, W]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        #x.shape =[32, 64, 64]

        x = self.layer1(x)
        #x.shape =[128, 64, 64]
        x = self.layer2(x)
        #x.shape =[256, 32, 32]
        x = self.layer3(x)
        #x.shape =[512, 16, 16]
        x = self.layer4(x)
        #x.shape =[1024, 8, 8]
        
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x
     

In [ ]:
resnet50 = ResNet(resnet.Bottleneck, [3,4,6,3],12, True).to(device)
# 1(conv1) + 9(layer1) + 12(layer2) + 18(layer3) + 9(layer4) +1(fc)= ResNet50

In [ ]:
!pip install torchsummary

In [ ]:
from torchsummary import summary
summary(resnet50, input_size=(3, 224, 224), device=device)

In [ ]:
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()
     
# 첫 번째 layer의 filter를 확인해보자 (=가중치 확인) -> 나중에 학습을 완료한 후의 filter도 확인하기
for w in resnet50.parameters():
    w = w.data.cpu()
    print(w.shape)
    break

# 가중치 renormalization
min_w = torch.min(w)
w1 = (-1/(2 * min_w)) * w + 0.5

# make grid to display it
grid_size = len(w1)
x_grid = [w1[i] for i in range(grid_size)]
x_grid = torchvision.utils.make_grid(x_grid, nrow=6, padding=1)

plt.figure(figsize=(10, 10))
imshow(x_grid)

In [ ]:
# config 모델 파라미터 인자를 만들기 위한 클래스
class Config:
    def __init__(self, **kwargs):
        for key, value in kwargs.items():
            setattr(self, key, value)
            
lr = 0.0008
epochs = 30
optimizer = 'Adam'

# 파라미터 클래스
config = Config(
    trainloader = train_loader,
    testloader = test_loader,
    validloader = valid_loader,
    model = resnet50,
    device = device,
    optimizer = torch.optim.Adam(resnet50.parameters(), lr=lr),
    criterion= nn.CrossEntropyLoss(class_weights).to(device),#class_weights
    globaliter = 0,
    patience=10
)

In [ ]:
from collections import defaultdict

train_log = defaultdict(list)
valid_log = defaultdict(list)
test_log = defaultdict(list)

class train_test():
    def __init__(self, config):
        # 파라미터 인자
        self.trainloader = config.trainloader
        self.testloader = config.testloader
        self.validloader = config.validloader
        self.model = config.model
        self.device = config.device
        self.optimizer = config.optimizer
        self.criterion = config.criterion
        self.globaliter = config.globaliter
        self.patience = config.patience
        print(len(self.trainloader))
        
    def train(self, epochs, log_interval, patience=10):
        self.model.train()
        best_loss = float('inf')
        no_improvement = 0
    
        for epoch in range(1, epochs + 1):  
            running_loss = 0.0
            running_loss_list = []
            lr_sche.step()
    
            for i, data in enumerate(self.trainloader, 0):
                self.globaliter += 1
                inputs, labels = data 
                inputs = inputs.to(self.device)
                labels = labels.to(self.device)
    
                self.optimizer.zero_grad() 
                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()
                running_loss += loss.item()
    
                if i % log_interval == log_interval - 1:
                    print('Train Epoch: {} [{}/{} ({:.0f}%)]\tlearningLoss: {:.6f}\twhole_loss: {:.6f} '.format(
                        epoch, i*len(inputs), len(self.trainloader.dataset),
                        100. * i*len(inputs) / len(self.trainloader.dataset), 
                        running_loss / log_interval,
                        loss.item()))
                    train_log['loss'].append(running_loss)
                    running_loss = 0.0

            # Validation
            with torch.no_grad():
                self.model.eval()
                valid_correct = 0
                valid_total = 0
                valid_loss = 0
                for data in self.validloader:
                    images, labels = data
                    images = images.to(self.device)
                    labels = labels.to(self.device)
                    outputs = self.model(images)
                    _, predicted = torch.max(outputs.data, 1)
                    valid_total += labels.size(0)
                    valid_correct += (predicted == labels).sum().item()
                    valid_loss += self.criterion(outputs, labels).item()

                avg_valid_loss = valid_loss / len(self.validloader)
                print('\n{} valid set : Average loss:{:.4f}, Accuracy: {}/{}({:.0f}%)'.format(
                      epoch, avg_valid_loss, valid_correct, valid_total, 100 * valid_correct/valid_total))
                valid_log['loss'].append(avg_valid_loss)
                valid_log['acc'].append(100 * valid_correct/valid_total)
                
            with torch.no_grad():
                self.model.eval()
                correct = 0
                total = 0
                test_loss = 0
                for data in self.testloader:
                    images, labels = data
                    images = images.to(self.device)
                    labels = labels.to(self.device)
                    outputs = self.model(images)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
                    test_loss += self.criterion(outputs, labels).item()

                avg_loss = test_loss / len(self.testloader)
                print('{} Test set : Average loss:{:.4f}, Accuracy: {}/{}({:.0f}%)\n'.format(
                      epoch, avg_loss, correct, total, 100 * correct/total))
                test_log['loss'].append(avg_loss)
                test_log['acc'].append(100 * correct/total)

                if avg_valid_loss < best_loss:
                    best_loss = avg_valid_loss
                    no_improvement = 0
                    torch.save(self.model.state_dict(), 'best_model.pt')
                else:
                    no_improvement += 1
                    if no_improvement >= self.patience:
                        print("Early stopping. No improvement for {} epochs.".format(self.patience))
                        torch.save(self.model.state_dict(), 'earlystop_model.pt')
                        return

        print('Finished Training')

In [ ]:
ready_to_train=train_test(config)
lr_sche = optim.lr_scheduler.StepLR(config.optimizer, step_size=100, gamma=0.5) # 20 step마다 lr조정
epochs = 100
log_interval = 50

# ready_to_train.train(epochs, log_interval)

In [ ]:
############################################################
#중간

In [ ]:
from torchvision import models

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = models.resnet50(pretrained=True).to(device)

num = 0
for param in model.parameters():
    num += 1

count = 0
for param in model.parameters():
    if count > 50:
        break
    param.requires_grad = False
    count += 1
    
model.fc = nn.Sequential(
    nn.Linear(2048, 128),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(128, 12),
    nn.LogSoftmax(dim=1)
)
model = model.to(device)

## visualize

In [ ]:
!mkdir filters
!mkdir filters/train
!mkdir filters/notrain

In [ ]:
model_weights = [] # we will save the conv layer weights in this list
conv_layers = [] # we will save the 49 conv layers in this list
# get all the model children as list
model_children = list(model.children())

# counter to keep count of the conv layers
counter = 0 
# append all the conv layers and their respective weights to the list
for i in range(len(model_children)):
    if type(model_children[i]) == nn.Conv2d:
        counter += 1
        model_weights.append(model_children[i].weight)
        conv_layers.append(model_children[i])
    elif type(model_children[i]) == nn.Sequential:
        for j in range(len(model_children[i])):
            for child in model_children[i][j].children():
                if type(child) == nn.Conv2d:
                    counter += 1
                    model_weights.append(child.weight)
                    conv_layers.append(child)
print(f"Total convolutional layers: {counter}")

# # take a look at the conv layers and the respective weights
# for weight, conv in zip(model_weights, conv_layers):
#     # print(f"WEIGHT: {weight} \nSHAPE: {weight.shape}")
#     print(f"CONV: {conv} ====> SHAPE: {weight.shape}")
    
# visualize the first conv layer filters
plt.figure(figsize=(20, 17))
for i, filter in enumerate(model_weights[0]):
    plt.subplot(8, 8, i+1) # (8, 8) because in conv0 we have 7x7 filters and total of 64 (see printed shapes)
#     plt.imshow(filter[0, :, :].detach())
    plt.axis('off')
    plt.savefig('./filters/notrain/filter.png')
# plt.show()

In [ ]:
import numpy as np
import torchvision.transforms as transforms

# Read and visualize an image
img = cv2.imread('/kaggle/working/image_dir/Surrealism/Salvador_Dali_139.jpg')
img = img.astype(np.uint8)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.show()

# Define the transforms
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
])

# Apply the transforms
img = transform(img)
print(img.size())

# Unsqueeze to add a batch dimension
img = img.unsqueeze(0)
print(img.size())

# Move the image tensor to GPU
img = img.to(device)

# Pass the image through all the layers
results = [conv_layers[0](img)]
for i in range(1, len(conv_layers)):
    # Pass the result from the last layer to the next layer
    results.append(conv_layers[i](results[-1]))

# Make a copy of the `results`
outputs = results

for num_layer in range(len(outputs)):
    plt.figure(figsize=(30, 30))
    layer_viz = outputs[num_layer][0, :, :, :]
    layer_viz = layer_viz.data
    print(layer_viz.size())
    for i, filter in enumerate(layer_viz):
        if i == 64: # we will visualize only 8x8 blocks from each layer
            break
        plt.subplot(8, 8, i + 1)
        plt.imshow(filter.cpu(), cmap='gray')  # Move tensor to CPU for visualization
        plt.axis("off")
    print(f"Saving layer {num_layer} feature maps...")
    plt.savefig(f"./filters/notrain/layer_{num_layer}.png")
    plt.close()


In [ ]:
lr = 0.0001
epochs = 30
optimizer = 'Adam'

# 파라미터 클래스
config = Config(
    trainloader = train_loader,
    testloader = test_loader,
    validloader = valid_loader,
    model = model,
    device = device,
    optimizer = torch.optim.Adam(model.parameters(), lr=lr),
    criterion= nn.CrossEntropyLoss(class_weights).to(device),#class_weights
    globaliter = 0,
    patience = 5
)

In [ ]:
ready_to_train=train_test(config)
lr_sche = optim.lr_scheduler.StepLR(config.optimizer, step_size=100, gamma=0.5) # 20 step마다 lr조정
epochs = 100
log_interval = 10

ready_to_train.train(epochs, log_interval)

## visualize

In [ ]:
model_weights = [] # we will save the conv layer weights in this list
conv_layers = [] # we will save the 49 conv layers in this list
# get all the model children as list
model_children = list(model.children())

# counter to keep count of the conv layers
counter = 0 
# append all the conv layers and their respective weights to the list
for i in range(len(model_children)):
    if type(model_children[i]) == nn.Conv2d:
        counter += 1
        model_weights.append(model_children[i].weight)
        conv_layers.append(model_children[i])
    elif type(model_children[i]) == nn.Sequential:
        for j in range(len(model_children[i])):
            for child in model_children[i][j].children():
                if type(child) == nn.Conv2d:
                    counter += 1
                    model_weights.append(child.weight)
                    conv_layers.append(child)
print(f"Total convolutional layers: {counter}")

# # take a look at the conv layers and the respective weights
# for weight, conv in zip(model_weights, conv_layers):
#     # print(f"WEIGHT: {weight} \nSHAPE: {weight.shape}")
#     print(f"CONV: {conv} ====> SHAPE: {weight.shape}")
    
# visualize the first conv layer filters
plt.figure(figsize=(20, 17))
for i, filter in enumerate(model_weights[0]):
    plt.subplot(8, 8, i+1) # (8, 8) because in conv0 we have 7x7 filters and total of 64 (see printed shapes)
#     plt.imshow(filter[0, :, :].detach())
    plt.axis('off')
    plt.savefig('./filters/train/filter.png')
plt.show()

In [ ]:
import numpy as np
import torchvision.transforms as transforms

# Read and visualize an image
img = cv2.imread('/kaggle/working/image_dir/Surrealism/Salvador_Dali_139.jpg')
img = img.astype(np.uint8)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.show()

# Define the transforms
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
])

# Apply the transforms
img = transform(img)
print(img.size())

# Unsqueeze to add a batch dimension
img = img.unsqueeze(0)
print(img.size())

# Move the image tensor to GPU
img = img.to(device)

# Pass the image through all the layers
results = [conv_layers[0](img)]
for i in range(1, len(conv_layers)):
    # Pass the result from the last layer to the next layer
    results.append(conv_layers[i](results[-1]))

# Make a copy of the `results`
outputs = results

for num_layer in range(len(outputs)):
    plt.figure(figsize=(30, 30))
    layer_viz = outputs[num_layer][0, :, :, :]
    layer_viz = layer_viz.data
    print(layer_viz.size())
    for i, filter in enumerate(layer_viz):
        if i == 64: # we will visualize only 8x8 blocks from each layer
            break
        plt.subplot(8, 8, i + 1)
        plt.imshow(filter.cpu(), cmap='gray')  # Move tensor to CPU for visualization
        plt.axis("off")
    print(f"Saving layer {num_layer} feature maps...")
    plt.savefig(f"./filters/train/layer_{num_layer}.png")
    plt.close()


In [ ]:
#######################################################
#최종

In [ ]:
from torchvision import models

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = models.resnet50(pretrained=True).to(device)

model.fc = nn.Sequential(
    nn.Linear(2048, 128),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(128, 12),
    nn.LogSoftmax(dim=1)
)

count = 0
for param in model.parameters():
    if count > 0:
        break
    param.requires_grad = False
    count += 1
    
model = model.to(device)

In [ ]:
lr = 0.00008
epochs = 30
optimizer = 'Adam'

# 파라미터 클래스
config = Config(
    trainloader = train_loader,
    testloader = test_loader,
    validloader = valid_loader,
    model = model,
    device = device,
    optimizer = torch.optim.Adam(model.parameters(), lr=lr),
    criterion= nn.CrossEntropyLoss(class_weights).to(device),#class_weights
    globaliter = 0,
    patience=10
)

In [ ]:
ready_to_train=train_test(config)
lr_sche = optim.lr_scheduler.StepLR(config.optimizer, step_size=100, gamma=0.5) # 20 step마다 lr조정
epochs = 100
log_interval = 10

#ready_to_train.train(epochs, log_interval)

In [ ]:
from IPython.display import FileLink
plt.plot(range(len(train_log['loss'])),train_log['loss']) 
plt.savefig('train_loss.png')
# 다운로드 링크 생성
FileLink('train_loss.png')

In [ ]:
plt.plot(test_log['loss'], label='Test loss')
plt.plot(valid_log['loss'], label='Validation loss')

# 그래프 제목 및 레이블 설정
plt.title('Testing, and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')

# 범례 표시
plt.legend()

plt.savefig('testvalid_loss.png')
# 다운로드 링크 생성
FileLink('testvalid_loss.png')

In [ ]:
plt.plot(test_log['acc'], label='Test acc')
plt.plot(valid_log['acc'], label='Validation acc')

# 그래프 제목 및 레이블 설정
plt.title('Testing, and Validation acc')
plt.xlabel('Epochs')
plt.ylabel('acc')

# 범례 표시
plt.legend()

plt.savefig('testvalid_acc.png')
# 다운로드 링크 생성
FileLink('testvalid_acc.png')